In [1]:
import numpy as np
import torch 
from torch import nn,optim
import torch.nn.functional as F
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices,char_to_idx,idx_to_char,vocab_size)=d2l.load_data_jay_lyrics()

In [2]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size
print('will use',device)

def get_params():
    def _one(shape):
        ts=torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    def _three():
        return (_one((num_inputs,num_hiddens)),
               _one((num_hiddens,num_hiddens)),
               torch.nn.Parameter(torch.zeros(num_hiddens,device=device,dtype=torch.float32),requires_grad=True)
               )
    
    W_xz,W_hz,b_z=_three()
    W_xr,W_hr,b_r=_three()
    W_xh,W_hh,b_h=_three()
    
    W_hq=_one((num_hiddens,num_outputs))
    b_q=torch.nn.Parameter(torch.zeros(num_outputs,device=device,dtype=torch.float32),requires_grad=True)
    return nn.ParameterList([W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh,W_hh,b_h,W_hq,b_q])


will use cuda


In [3]:
def init_gru_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device),)

In [4]:
def gru(inputs,state,params):
    W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh,W_hh,b_h,W_hq,b_q=params
    H,=state
    outputs=[]
    for X in inputs:
        Z=torch.sigmoid(torch.matmul(X,W_xz)+torch.matmul(H,W_hz)+b_z)
        R=torch.sigmoid(torch.matmul(X,W_xr)+torch.matmul(H,W_hr)+b_r)
        H_tilda=torch.tanh(torch.matmul(X,W_xh)+torch.matmul(R*H,W_hh)+b_h)
        H=Z*H+(1-Z)*H_tilda
        Y=torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
        
    return outputs,(H,)

In [5]:
num_epochs,num_steps,batch_size,lr,clipping_theta=160,35,32,1e2,1e-2
pred_period,pred_len,prefixes=40,50,['分开','不分开']

In [6]:
d2l.train_and_predict_rnn(gru,get_params,init_gru_state,num_hiddens,vocab_size,
                         device,corpus_indices,idx_to_char,char_to_idx,
                         False,num_epochs,num_steps,lr,clipping_theta,batch_size,
                         pred_period,pred_len,prefixes)

epoch 40,perplexity 152.906348,time 1.64 sec
 - 分开 我想你 我不不 我想你 你不了 我不不 你不不 我想你 你不了 我不不 你不不 我想你 你不了 我
 - 不分开 我想你 我不不 我想你 你不了 我不不 你不不 我想你 你不了 我不不 你不不 我想你 你不了 我
epoch 80,perplexity 34.288970,time 1.52 sec
 - 分开 我想要这样 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
epoch 120,perplexity 6.087475,time 1.64 sec
 - 分开 一直心酒 再来一碗热粥 配上几斤的牛肉 我说店小二 三两银够不够 景色入秋 漫天黄沙凉过 塞北的客
 - 不分开  没有你在我有多难熬多  说  有去共渡每堡 说有去了南棍 他使在双截棍 哼哼哈兮 快使用双截棍 
epoch 160,perplexity 1.780528,time 1.59 sec
 - 分开 那小的梦笑 喜欢一直 何伤了用点棍花 所有苦愿里里去 我 想和你飞棒堡 我想想你的微笑每天都能看到
 - 不分开  没有你在我有多难熬多难熬  穿过云层 我试著努力向你奔跑 爱才送到 你却已在别人怀抱 就是开不了


In [9]:
lr=1e-2
gru_layer=nn.GRU(input_size=vocab_size,hidden_size=num_hiddens)
model=d2l.RNNModel(gru_layer,vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,device,
                                 corpus_indices,idx_to_char,char_to_idx,
                                 num_epochs,num_steps,lr,clipping_theta,
                                 batch_size,pred_period,pred_len,prefixes)

NameError: name 'num_epochs' is not defined